In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
sys.path.append('/Users/gracecolverd/New_dataset')
from src import * 

In [3]:
import os
import sys
import time
import geopandas as gpd 
import pandas as pd 
import glob 
import re
import matplotlib.pyplot as plt 
from shapely.geometry import Point, Polygon
from shapely.geometry import box
sys.path.append('/Users/gracecolverd/New_dataset')

from src.postcode_utils import find_postcode_for_ONSUD_file, find_data_pc

from src.multi_thread import run_batching, merge_temp_logs_to_main , generate_batch_list    

# from src import process_postcode, postcode_median_age_batch_fn , run_batching, find_data_pc
# from src import calculate_modal_age_band, calc_med_attr , postcode_modal_batch_fn, gen_postcode_areas
# from src import postcode_area_vars_batch_fn , fill_premise_floor_types


In [4]:
# from osgeo import ogr

# # Path to your GeoPackage file
# gpkg_path = '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_full_building_data/6101/Data/UKBuildings_Edition_15_new_format.gpkg'

# # Open the GeoPackage file
# ds = ogr.Open(gpkg_path, 0)  # 0 means read-only mode

# if ds is None:
#     print("Could not open the GeoPackage file.")
# else:
#     # Assuming we're interested in the first layer
#     # If you need a specific layer, you can use ds.GetLayerByName('layer_name')
#     layer = ds.GetLayerByIndex(0)
   
    
#     if layer is None:
#         print("Could not access the layer.")
#     else:
#         # Get the layer definition which holds information about the fields
#         layer_defn = layer.GetLayerDefn()
#         for feature in layer:
#             feat_data = feature.items()
#             # Optionally, add the feature ID if you want to use it similarly to an index
#             feat_data['feature_id'] = feature.GetFID()
#         # Loop through the field definitions and print the field names
#         column_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]
#         print("Column names:", column_names)

# # Close the dataset
# ds = None


In [ ]:
gpkg_path = '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_full_building_data/6101/Data/UKBuildings_Edition_15_new_format.gpkg'



In [34]:
lin = open('/Users/gracecolverd/New_dataset/old/testing/batches/EE/batch_9.txt', 'r' )

In [4]:
build = gpd.read_file('/Volumes/T9/postcode_data/data/verisk_y2022_V4/bbox_529252.36_186435.25920425786_539252.36_196435.25920425786.gpkg')


In [5]:
build.premise_age.unique()

array(['1980-1989', '1945-1959,1980-1989', 'Post 1999', 'Unknown date',
       '1960-1979', '1945-1959', '1919-1944', 'Pre 1919', '1837-1869',
       '1990-1999', '1870-1918', '1945-1959,Unknown date',
       '1919-1944,1980-1989', '1980-1989,Pre 1919,Unknown date',
       '1980-1989,Unknown date', 'Pre 1837', '1945-1959,Pre 1919',
       '1919-1944,1945-1959,Pre 1919', '1945-1959,1980-1989,Pre 1919',
       '1980-1989,Pre 1919', '1919-1944,1945-1959', '1919-1944,Pre 1919',
       'Pre 1919,Unknown date', '1919-1944,1945-1959,1980-1989',
       '1945-1959,1980-1989,Unknown date',
       '1919-1944,Pre 1919,Unknown date',
       '1919-1944,1945-1959,1980-1989,Pre 1919', '1919-1944,Unknown date',
       '1945-1959,1980-1989,Pre 1919,Unknown date',
       '1919-1944,1945-1959,1980-1989,Unknown date',
       '1919-1944,1945-1959,1980-1989,Pre 1919,Unknown date',
       '1919-1944,1945-1959,Unknown date',
       '1919-1944,1945-1959,Pre 1919,Unknown date',
       '1919-1944,1980-1989,Pre 19

In [6]:
build[build['map_simple_use']=='Residential'].premise_age.unique()  

array(['1980-1989', 'Post 1999', '1960-1979', 'Unknown date', '1945-1959',
       '1837-1869', '1990-1999', '1870-1918', '1919-1944', 'Pre 1837',
       'Pre 1919'], dtype=object)

In [8]:
build.premise_type.unique().tolist() 

['Medium height flats 5-6 storeys',
 None,
 'Small low terraces',
 '3-4 storey and smaller flats',
 'Domestic outbuilding',
 'Tall terraces 3-4 storeys',
 'Large semi detached',
 'Standard size detached',
 'Standard size semi detached',
 '2 storeys terraces with t rear extension',
 'Semi type house in multiples',
 'Tall flats 6-15 storeys',
 'Large detached',
 'Very tall point block flats',
 'Very large detached',
 'Planned balanced mixed estates',
 'Linked and step linked premises']

In [7]:
one_hot = pd.get_dummies(build['premise_type'])
build2 = pd.concat([build, one_hot], axis=1)


In [11]:
build.columns

Index(['premise_age', 'premise_year', 'premise_use', 'premise_type',
       'premise_floor_count', 'height', 'premise_area', 'building_area',
       'address_area', 'gross_area', 'basement', 'listed_grade',
       'element_count', 'bathroom_count', 'bedroom_count',
       'reception_room_count', 'roof_type', 'wall_type', 'substructure_type',
       'glazing_type', 'wall_construction_type', 'extension_count',
       'habitable_rooms', 'open_fireplaces', 'floor_type', 'distance_building',
       'site_id', 'site_area', 'site_non_built_area', 'site_building_count',
       'distance_water', 'verisk_building_id', 'uprn', 'uprn_count',
       'uprn_distance', 'toid', 'map_age', 'map_floors', 'map_use',
       'map_simple_use', 'geometry'],
      dtype='object')

In [26]:
import pandas as pd
from scipy.stats import mode

# Assuming 'build' is your DataFrame
# Define a custom aggregation function that returns the first mode in case of multiple modes
def agg_mode(x):
    # check if multiple modes 
    # if len(mode(x).mode) > 1:
    #     # return the first mode
    #     print('multiple modes found') 
    return mode(x).mode[0]

# Use the custom function with groupby. Note that 'apply' is used to accommodate custom logic
mode_result = build.groupby(['premise_use', 'premise_type'])['premise_age'].apply(agg_mode).reset_index(name='mode_age')

print(mode_result)


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_25760/1358165263.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(x).mode[0]
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_25760/1358165263.py:11: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.Data

               premise_use                  premise_type   mode_age
0   Commercial - mixed use  3-4 storey and smaller flats  1945-1959
1   Commercial - mixed use            Small low terraces   Pre 1919
2   Commercial - mixed use        Standard size detached   Pre 1919
3   Commercial - mixed use   Standard size semi detached  1919-1944
4   Commercial - mixed use     Tall terraces 3-4 storeys  1945-1959
..                     ...                           ...        ...
78            Unclassified     Tall terraces 3-4 storeys  1980-1989
79               Utilities  3-4 storey and smaller flats  1919-1944
80               Utilities            Small low terraces  1945-1959
81               Utilities        Standard size detached  1945-1959
82               Utilities   Standard size semi detached  1980-1989

[83 rows x 3 columns]


In [14]:
build.groupby(['map_simple_use', 'premise_age', 'premise_floor_count'])['height'].mean()

map_simple_use  premise_age   premise_floor_count
Mixed Use       1870-1918     3                      14.050000
                1919-1944     1                       9.660000
                              1,2                    12.400000
                              1,3                     8.900000
                              2                      10.636235
                                                       ...    
Residential     Pre 1919      3                       8.287500
                              4                      12.266667
                              5                      15.200000
                Unknown date  1                       3.000034
                              2                       3.000000
Name: height, Length: 463, dtype: float64

In [ ]:

# Compute the weighted average
weighted_avg = aggregated_stats.div(weights['count'], axis=0)

In [29]:
def one_hot_encode_premise_type(df):
    pre_cols = df.columns.tolist()
    one_hot = pd.get_dummies(df['premise_type'], 'premusetype')
    df = pd.concat([df, one_hot], axis=1)
    post_cols = df.columns.tolist()
    new_cols = [col for col in post_cols if col not in pre_cols]
    return df, new_cols 

In [30]:
df, n = one_hot_encode_premise_type(build)

In [33]:
ll = open('/Users/gracecolverd/New_dataset/old/testing/batches/EE/batch_9.txt', 'r')

In [15]:
pre = build.columns.tolist()
post = build2.columns.tolist()
# cols to sum : 
cols_to_sum = [x for x in post if x not in pre]
cols_to_sum

['2 storeys terraces with t rear extension',
 '3-4 storey and smaller flats',
 'Domestic outbuilding',
 'Large detached',
 'Large semi detached',
 'Linked and step linked premises',
 'Medium height flats 5-6 storeys',
 'Planned balanced mixed estates',
 'Semi type house in multiples',
 'Small low terraces',
 'Standard size detached',
 'Standard size semi detached',
 'Tall flats 6-15 storeys',
 'Tall terraces 3-4 storeys',
 'Very large detached',
 'Very tall point block flats']

In [17]:
# sum cols
build2[cols_to_sum].sum()

2 storeys terraces with t rear extension    55407
3-4 storey and smaller flats                 6460
Domestic outbuilding                        46760
Large detached                                249
Large semi detached                          3137
Linked and step linked premises                31
Medium height flats 5-6 storeys               497
Planned balanced mixed estates                157
Semi type house in multiples                27874
Small low terraces                          40372
Standard size detached                       5678
Standard size semi detached                 30352
Tall flats 6-15 storeys                        87
Tall terraces 3-4 storeys                    8046
Very large detached                             8
Very tall point block flats                    89
dtype: int64

In [ ]:
building_groupings = {
    "Low-Rise Residential": [
        "Small low terraces",
        "Domestic outbuilding",
        "Standard size semi-detached",
        "Standard size detached",
        "Large semi-detached",
        "2 storeys terraces with t rear extension"
         "Very large detached", 
          "Large detached",
    ],
    "Medium-Rise Residential": [
        "Medium height flats 5-6 storeys",
        "3-4 storey and smaller flats",
        "Tall terraces 3-4 storeys",
        "Semi-type house in multiples"
    ],
    "High-Rise Residential": [
        "Tall flats 6-15 storeys",
        "Very tall point block flats",
       
    ],
    "Special Categories": [
        "Large detached",
        "Planned balanced mixed estates",
        "Linked and step linked premises"
    ]
}

print(building_groupings)


In [151]:
build[build['premise_floor_count'] =='1,2'].to_csv('test_builds.csv')

In [152]:
build['height'].describe() 

count    248585.000000
mean          8.908354
std           4.127153
min           0.000000
25%           5.800000
50%           9.900000
75%          11.000000
max         142.700000
Name: height, dtype: float64

In [20]:
build[build['height']==0].premise_age.value_counts()


Post 1999       7055
Unknown date    2918
1870-1918        914
1919-1944        824
1945-1959        352
1960-1979        307
1990-1999        173
1980-1989        172
Pre 1919          95
Pre 1837          13
1837-1869          8
Name: premise_age, dtype: int64

In [24]:
df = build
df['premise_age_bucketed'] = np.where(df['premise_age'].isin(['Pre 1837', '1837-1869', '1870-1918']), 'Pre 1919', df['premise_age']) 

In [28]:
df.premise_age_bucketed.unique()

array(['Unknown date', 'Pre 1919', '1919-1944', '1960-1979', '1980-1989',
       'Post 1999', '1945-1959', '1990-1999', '1945-1959,Pre 1919',
       '1980-1989,Pre 1919', '1945-1959,Post 1999',
       '1945-1959,Unknown date', '1960-1979,Unknown date',
       '1945-1959,1980-1989,Unknown date', '1980-1989,Unknown date',
       '1919-1944,Pre 1919', '1945-1959,1980-1989',
       '1945-1959,1980-1989,Pre 1919', '1919-1944,1945-1959',
       '1919-1944,1980-1989', '1919-1944,1945-1959,1990-1999',
       '1919-1944,1960-1979', '1990-1999,Pre 1919',
       'Post 1999,Unknown date', '1945-1959,Post 1999,Unknown date',
       '1990-1999,Unknown date', '1919-1944,Unknown date',
       'Pre 1919,Unknown date', '1945-1959,Pre 1919,Unknown date',
       '1945-1959,1990-1999', '1919-1944,1945-1959,1980-1989',
       '1945-1959,1990-1999,Pre 1919', '1990-1999,Pre 1919,Unknown date',
       '1919-1944,Post 1999', '1919-1944,1945-1959,Pre 1919',
       '1919-1944,1990-1999,Pre 1919', '1919-1944,Pre 1

In [40]:

def get_val_from_age_band(row):
    age_band = row['premise_age']
    if age_band == 'Pre 1919':
        val = 3
    elif age_band == '1919-1944':
        val = 2.9
    elif age_band == '1945-1959':
        val = 2.8
    elif age_band == '1960-1979':
        val = 2.2
    elif age_band == '1980-1989':
        val = 2.2
    elif age_band == '1990-1999':
        val = 2.2
    elif age_band == 'Post 1999':
        val = 2.4
    else:
        val = 2.3
    return val

    # Use this function to get the 'val' based on 'age_band'
    # val = get_val_from_age_band(age_band)

def fill_missing_height(df):
    for index, row in df.iterrows():
            if row['height'] == 0:
                val = get_val_from_age_band(row)
                df.at[index, 'height'] = (int(row['premise_floor_count']) * val) * 1.15 

    return df

In [79]:
df = build[build['premise_use']=='Residential'].copy() 

In [54]:
df.columns

Index(['premise_age', 'premise_year', 'premise_use', 'premise_type',
       'premise_floor_count', 'height', 'premise_area', 'building_area',
       'address_area', 'gross_area', 'basement', 'listed_grade',
       'element_count', 'bathroom_count', 'bedroom_count',
       'reception_room_count', 'roof_type', 'wall_type', 'substructure_type',
       'glazing_type', 'wall_construction_type', 'extension_count',
       'habitable_rooms', 'open_fireplaces', 'floor_type', 'distance_building',
       'site_id', 'site_area', 'site_non_built_area', 'site_building_count',
       'distance_water', 'verisk_building_id', 'uprn', 'uprn_count',
       'uprn_distance', 'toid', 'map_age', 'map_floors', 'map_use',
       'map_simple_use', 'geometry', 'premise_age_bucketed'],
      dtype='object')

In [55]:
df.map_simple_use.value_counts()

Residential        291361
Non Residential     81177
Mixed Use            1996
Name: map_simple_use, dtype: int64

In [103]:
link = pd.read_csv('/Volumes/T9/Data_downloads/Versik_building_data/2024_03_full_building_data/6101/Data/edition_15_abc_link_file.csv') 

In [4]:
import geopandas as gpd 

df = gpd.read_file('/Volumes/T9/postcode_data/data/verisk_y2022_V5/bbox_69252.359375_816435.2583007812_79252.359375_826435.2583007812.gpkg')


In [9]:
import pandas as pd 
df = pd.read_csv('/Users/gracecolverd/New_dataset/proc/processing_log.csv', header=None) 


,0,1,2,3
0,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(9252.359375, 6435.25830078125, 19252.359375, ...",completed,0
1,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(9252.359375, 16435.25830078125, 19252.359375,...",completed,0
2,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(9252.359375, 26435.25830078125, 19252.359375,...",completed,0
3,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(9252.359375, 36435.25830078125, 19252.359375,...",completed,0
4,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(9252.359375, 46435.25830078125, 19252.359375,...",completed,0
...,...,...,...,...
1630,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(139252.359375, 446435.25830078125, 149252.359...",completed,0
1631,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(139252.359375, 456435.25830078125, 149252.359...",completed,0
1632,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(139252.359375, 466435.25830078125, 149252.359...",completed,0
1633,/Volumes/T9/postcode_data/data/verisk_y2022_V5...,"(139252.359375, 476435.25830078125, 149252.359...",completed,0


In [19]:
import os 
import pandas as pd 
def merge_temp_logs_to_main(log_file="/Users/gracecolverd/New_dataset/proc/log2.csv"):
    """
    Merge all temporary log files created by threads into the main log file.

    Parameters:
    - log_file: str, the path to the main log file.
    """
    # Assume temp files are in the same directory as the main log file
    temp_files = [f for f in os.listdir(os.path.dirname(log_file)) if f.startswith('temp_log_') and f.endswith('.csv')]
    print('num of temp files found ', len(temp_files) ) 
    # Create or append to the main log file
    if len(temp_files) != 0:
        for temp_file in temp_files:
            temp_file_path = os.path.join(os.path.dirname(log_file), temp_file)
            df_temp = pd.read_csv(temp_file_path )
            
            if os.path.exists(log_file):
                df_temp.to_csv(log_file, mode='a', header=False, index=False)
            else:
                df_temp.to_csv(log_file, mode='w', header=True, index=False)
    print('fn complete')

In [20]:
merge_temp_logs_to_main()

num of temp files found  6
fn complete


In [23]:
df = gpd.read_file('/Volumes/T9/postcode_data/data/verisk_y2022_V5/bbox_199252.359375_786435.2583007812_209252.359375_796435.2583007812.gpkg')

In [24]:
l

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,upn,geometry
0,1870-1918,NaN,Recreation and leisure,None,None,19.3,78.8,78.8,NaN,NaN,...,130167157.0,1,0.0,osgb1000039285271,1870-1918,None,Recreation and leisure,Non Residential,3053015,"MULTIPOLYGON (((200156.00000 796065.50000, 200..."
1,Unknown date,NaN,Residential,None,2,7.9,131.9,131.9,NaN,263.8,...,NaN,0,NaN,None,Unknown date,2,Residential,Residential,28888619,"MULTIPOLYGON (((200214.18986 796114.43163, 200..."
2,Unknown date,NaN,Residential,None,2,9.0,59.0,59.0,NaN,118.0,...,NaN,0,NaN,None,Unknown date,2,Residential,Residential,28889645,"MULTIPOLYGON (((203075.27780 792617.29457, 203..."
3,Unknown date,NaN,Residential,None,2,8.6,287.4,287.4,NaN,574.8,...,NaN,0,NaN,None,Unknown date,2,Residential,Residential,28889647,"MULTIPOLYGON (((203320.01347 792579.51641, 203..."
4,Unknown date,NaN,Residential,None,2,8.2,132.4,132.4,NaN,264.8,...,NaN,0,NaN,None,Unknown date,2,Residential,Residential,28889709,"MULTIPOLYGON (((205370.56000 792354.29000, 205..."
5,Post 1999,2000.0,Residential,Large detached,2,8.4,170.1,170.1,340.2,340.2,...,130073603.0,1,0.0,osgb1000039285275,Post 1999,2,Residential,Residential,2515927,"MULTIPOLYGON (((201053.50000 792316.00000, 201..."
6,Pre 1919,NaN,Residential,Standard size detached,2,8.7,100.1,100.1,200.2,200.2,...,130073602.0,1,0.0,osgb1000039285274,Pre 1919,2,Residential,Residential,22142161,"MULTIPOLYGON (((206327.00000 792251.00000, 206..."
7,Unknown date,NaN,Residential,None,2,8.6,52.2,52.2,NaN,104.4,...,NaN,0,NaN,None,Unknown date,2,Residential,Residential,2493373,"MULTIPOLYGON (((206325.35000 792273.54000, 206..."


In [2]:
import pandas as pd 
df = pd.read_csv('/Users/gracecolverd/New_dataset/proc/processing_log.csv')


In [4]:
build

NameError: name 'build' is not defined

In [111]:
merg = build.merge(link, left_on = ['verisk_building_id', 'toid'],  right_on = ['ubn', 'toid' ] )

In [226]:
build.height.describe()

count    248585.000000
mean          8.908354
std           4.127153
min           0.000000
25%           5.800000
50%           9.900000
75%          11.000000
max         142.700000
Name: height, dtype: float64

In [229]:
# Convert 'premise_floor_count' to numeric, coercing errors to NaN
height_bins = [0, 2 , 4 , 6, 10, 12, 14, 18, 20, 30 , 40 ,50 , 100 ,200 ]  # Example height ranges in meters
height_labels = [f"{height_bins[i]}-{height_bins[i+1]}m" for i in range(len(height_bins)-1)]


gdf = build.copy() 
gdf['premise_floor_count'] = pd.to_numeric(gdf['premise_floor_count'], errors='coerce')
gdf

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
0,1980-1989,NaN,Residential,Medium height flats 5-6 storeys,6.0,17.7,1350.4,3561.8,65.3,8102.4,...,33516887,1.001279e+10,124,0.0,osgb1000001802803021,1980-1989,6,Residential,Residential,"MULTIPOLYGON (((530812.730 186415.470, 530813...."
1,"1945-1959,1980-1989",NaN,Emergency services,None,2.0,8.1,655.2,655.2,655.2,1310.4,...,33516867,5.300051e+09,2,0.0,osgb1000005591431,Complex age,2,Emergency services,Non Residential,"MULTIPOLYGON (((530696.780 186413.130, 530694...."
2,Post 1999,NaN,Residential,Medium height flats 5-6 storeys,6.0,12.8,838.5,838.5,104.8,5031.0,...,33516874,1.001279e+10,48,0.0,osgb1000001802802966,Post 1999,6,Residential,Residential,"MULTIPOLYGON (((530739.220 186418.280, 530734...."
3,Unknown date,NaN,Unclassified,None,NaN,4.4,254.9,254.9,NaN,NaN,...,33516886,NaN,0,NaN,None,Unknown date,None,Unclassified,Non Residential,"MULTIPOLYGON (((530880.297 186434.031, 530884...."
4,1960-1979,NaN,Residential,Small low terraces,2.0,7.1,42.9,300.5,85.8,85.8,...,33516890,5.300040e+09,1,0.0,osgb1000005591451,1960-1979,2,Residential,Residential,"MULTIPOLYGON (((530739.510 186453.880, 530731...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248606,Unknown date,NaN,Residential,Domestic outbuilding,1.0,3.0,11.7,11.7,NaN,11.7,...,33932826,NaN,0,NaN,None,Unknown date,1,Residential,Residential,"MULTIPOLYGON (((533492.700 196404.480, 533489...."
248607,Unknown date,NaN,Residential,Domestic outbuilding,1.0,3.0,5.8,5.8,NaN,5.8,...,33932825,NaN,0,NaN,None,Unknown date,1,Residential,Residential,"MULTIPOLYGON (((533489.810 196399.540, 533489...."
248608,Unknown date,NaN,Residential,Domestic outbuilding,1.0,3.0,13.1,45.9,NaN,13.1,...,33932859,NaN,0,NaN,None,Unknown date,1,Residential,Residential,"MULTIPOLYGON (((533493.900 196418.760, 533491...."
248609,Unknown date,NaN,Residential,Domestic outbuilding,1.0,3.0,5.7,5.7,NaN,5.7,...,33932821,NaN,0,NaN,None,Unknown date,1,Residential,Residential,"MULTIPOLYGON (((533493.265 196398.770, 533493...."


In [235]:

# Exclude entries without valid height or premise_floor_count
valid_gdf = gdf.dropna(subset=['premise_floor_count']).query('height > 0')
valid_gdf['height_bucket'] = pd.cut(valid_gdf['height'], bins=height_bins, labels=height_labels, right=False)

valid_gdf.groupby(['premise_age', 'map_simple_use', 'height_bucket']).count()

premise_year  premise_use  \
premise_age  map_simple_use height_bucket                              
1837-1869    Mixed Use      0-2m                      0            0   
                            2-4m                      0            0   
                            4-6m                      0            0   
                            6-10m                     0            0   
                            10-12m                    0            0   
...                                                 ...          ...   
Unknown date Residential    20-30m                    0            0   
                            30-40m                    0            0   
                            40-50m                    0            0   
                            50-100m                   0            0   
                            100-200m                  0            0   

                                           premise_type  premise_floor_count  \
premise_age  map_simple_use height_bucket                                      
1837-1869    Mixed Use      0-2m                      0                    0   
                            2-4m                      0                    0   
                            4-6m                      0                    0   
                            6-10m                     0                    0   
                            10-12m                    0                    0   
...                                                 ...                  ...   
Unknown date Residential    20-30m                    0                    0   
                            30-40m                    0                    0   
                            40-50m                    0                    0   
                            50-100m                   0                    0   
                            100-200m                  0                    0   

                                           height  premise_area  \
premise_age  map_simple_use height_bucket                         
1837-1869    Mixed Use      0-2m                0             0   
                            2-4m                0             0   
                            4-6m                0             0   
                            6-10m               0             0   
                            10-12m              0             0   
...                                           ...           ...   
Unknown date Residential    20-30m              0             0   
                            30-40m              0             0   
                            40-50m              0             0   
                            50-100m             0             0   
                            100-200m            0             0   

                                           building_area  address_area  \
premise_age  map_simple_use height_bucket                                
1837-1869    Mixed Use      0-2m                       0             0   
                            2-4m                       0             0   
                            4-6m                       0             0   
                            6-10m                      0             0   
                            10-12m                     0             0   
...                                                  ...           ...   
Unknown date Residential    20-30m                     0             0   
                            30-40m                     0             0   
                            40-50m                     0             0   
                            50-100m                    0             0   
                            100-200m                   0             0   

                                           gross_area  basement  ...  \
premise_age  map_simple_use height_bucket                        ...   
1837-1869    Mixed Use      0-2m                    0         0  ...   
              

In [244]:
valid_gdf.groupby(['map_simple_use', 'height_bucket', 'premise_age']).count().sort_values('premise_floor_count')

premise_year  \
map_simple_use  height_bucket premise_age                                   
Mixed Use       0-2m          1837-1869                                 0   
Non Residential 100-200m      1919-1944,1945-1959                       0   
                              1919-1944,1945-1959,1980-1989             0   
                              1919-1944,1960-1979                       0   
                              1919-1944,1980-1989                       0   
...                                                                   ...   
Residential     6-10m         1870-1918                                 2   
                10-12m        1919-1944                                 0   
                6-10m         1919-1944                                 0   
                2-4m          Unknown date                              0   
                10-12m        1870-1918                                13   

                                                             premise_use  \
map_simple_use  height_bucket premise_age                                  
Mixed Use       0-2m          1837-1869                                0   
Non Residential 100-200m      1919-1944,1945-1959                      0   
                              1919-1944,1945-1959,1980-1989            0   
                              1919-1944,1960-1979                      0   
                              1919-1944,1980-1989                      0   
...                                                                  ...   
Residential     6-10m         1870-1918                            19509   
                10-12m        1919-1944                            23229   
                6-10m         1919-1944                            28025   
                2-4m          Unknown date                         46771   
                10-12m        1870-1918                            47914   

                                                             premise_type  \
map_simple_use  height_bucket premise_age                                   
Mixed Use       0-2m          1837-1869                                 0   
Non Residential 100-200m      1919-1944,1945-1959                       0   
                              1919-1944,1945-1959,1980-1989             0   
                              1919-1944,1960-1979                       0   
                              1919-1944,1980-1989                       0   
...                                                                   ...   
Residential     6-10m         1870-1918                             19508   
                10-12m        1919-1944                             23229   
                6-10m         1919-1944                             28025   
                2-4m          Unknown date                          46760   
                10-12m        1870-1918                             47914   

                                                             premise_floor_count  \
map_simple_use  height_bucket premise_age                                          
Mixed Use       0-2m          1837-1869                                        0   
Non Residential 100-200m      1919-1944,1945-1959                              0   
                              1919-1944,1945-1959,1980-1989                    0   
                              1919-1944,1960-1979                              0   
                              1919-1944,1980-1989                              0   
...                                                                          ...   
Residential     6-10m         1870-1918                                    19509   
                10-12m        1919-1944                                    23229   
                6-10m         1919-1944                                    28025   
                2-4m          Unknown date                                 46771   
                10-12m        1870-1918                     

In [247]:
valid_gdf.groupby(['map_simple_use', 'height_bucket', 'premise_age']).mean()['premise_floor_count'].dropna()


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_86435/4031160283.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  valid_gdf.groupby(['map_simple_use', 'height_bucket', 'premise_age']).mean()['premise_floor_count'].dropna()


map_simple_use  height_bucket  premise_age
Mixed Use       2-4m           1919-1944       2.291667
                               1945-1959       2.882353
                               1980-1989       3.000000
                               Pre 1919        2.590164
                4-6m           1919-1944       2.571429
                                                ...    
Residential     50-100m        1945-1959      18.000000
                               1960-1979      10.380000
                               Post 1999      30.000000
                100-200m       1980-1989       4.000000
                               Post 1999      51.000000
Name: premise_floor_count, Length: 302, dtype: float64

In [248]:

grouped = valid_gdf.groupby(['map_simple_use', 'premise_age', 'height_bucket'])
avf_fl_count = grouped['premise_floor_count'].mean().rename('avg_floor_count')
counts = grouped.size().rename('count')
summary = pd.concat([avf_fl_count, counts], axis=1).reset_index()

In [250]:
summary.dropna(subset= ['avg_floor_count'])

,map_simple_use,premise_age,height_bucket,avg_floor_count,count
18,Mixed Use,1870-1918,12-14m,3.000000,1
19,Mixed Use,1870-1918,14-18m,3.000000,1
27,Mixed Use,1919-1944,2-4m,2.291667,24
28,Mixed Use,1919-1944,4-6m,2.571429,14
29,Mixed Use,1919-1944,6-10m,2.025000,120
...,...,...,...,...,...
914,Residential,Pre 1919,10-12m,2.050000,20
915,Residential,Pre 1919,12-14m,2.666667,3
916,Residential,Pre 1919,14-18m,3.250000,4
924,Residential,Unknown date,2-4m,1.000107,46771


In [ ]:

# Bucket the heights

print(valid_gdf[['height_bucket', 'premise_floor_count']])
grouped = valid_gdf.groupby(['map_simple_use', 'premise_age', 'height_bucket'])
print(grouped)
avf_fl_count = grouped['premise_floor_count'].mean().rename('avg_floor_count')
print(avf_fl_count)

In [208]:
import pandas as pd
import numpy as np

def fill_premise_floor_types(uprn):
    # Preprocess 'premise_floor_count' to handle '1,2' cases
    def handle_comma_separated_values(val):
    # Check if the value is explicitly None or an empty string
        if val is None or val == '':
            return np.nan

        if val in ['1,2', '2,3', '3,4' , '1,3', '4,5', '3,5','5,6', '4,6','5,7']:
            parts = [int(part) for part in val.split(',')]
            return np.mean(parts)


        try:
            return float(val)
        except ValueError:
            return np.nan



    print('starting floor replace')
    uprn['premise_floor_count'] = uprn['premise_floor_count'].apply(handle_comma_separated_values)
    
    # Compute 'av_storey_height' only for non-NaN 'premise_floor_count'
    valid_data = uprn.dropna(subset=['premise_floor_count']).copy()
    valid_data['av_storey_height'] = valid_data['height'] / valid_data['premise_floor_count']

    
    # Calculate mean storey height from valid entries
    mean_storey_height = valid_data['av_storey_height'].mean()
    
    # Fill NaN 'premise_floor_count' by dividing 'height' by mean storey height and taking floor to ensure it's never higher than heated volume
    uprn['premise_floor_count'].fillna(uprn['height'] / mean_storey_height, inplace=True)
    # uprn['premise_floor_count'] = np.floor(uprn['premise_floor_count']).astype('Int64')
    
    return uprn


In [213]:

res_use_types = ['Residential',  'Residential with retail below', 'Retail below office or residential' ] 

res = build[build['premise_use'].isin(res_use_types)].copy() 
res.premise_floor_count.value_counts()

2             165238
1              46776
3              10911
4               7593
5                609
6                393
2,3               44
3,4               31
7                 29
12                21
10                20
11                18
15                14
8                 12
13                11
14                10
16                 8
23                 6
18                 6
34                 6
17                 5
20                 4
24                 4
1,3                3
19                 3
22                 2
2,4                2
3,5                2
4,5                2
30                 2
3,7                1
51                 1
5,7                1
9                  1
3,8                1
10,3,4,6,8         1
Name: premise_floor_count, dtype: int64

In [214]:
res[res['premise_floor_count'] == '10,3,4,6,8']

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
173457,1980-1989,NaN,Retail below office or residential,None,"10,3,4,6,8",35.9,6372.2,6372.2,490.2,63722.0,...,33882222,207203033.0,130,0.0,osgb1000006216177,1980-1989,Complex floors,Retail below office or residential,Mixed Use,"MULTIPOLYGON (((534395.930 193527.970, 534389...."


In [210]:
test = fill_premise_floor_types(res)

starting floor replace


In [215]:
test[test['verisk_building_id']==33882222][test['premise_use']=='Retail below office or residential']

/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
173457,1980-1989,NaN,Retail below office or residential,None,7.876984,35.9,6372.2,6372.2,490.2,63722.0,...,33882222,207203033.0,130,0.0,osgb1000006216177,1980-1989,Complex floors,Retail below office or residential,Mixed Use,"MULTIPOLYGON (((534395.930 193527.970, 534389...."


In [218]:
res[res['premise_use']=='Residential'].premise_area.mean()

63.71450746348248

In [223]:
res[res['premise_use']=='Residential'].premise_area.describe()

count    225029.000000
mean         63.714507
std          79.538328
min           5.100000
25%          35.400000
50%          55.700000
75%          74.800000
max        4929.200000
Name: premise_area, dtype: float64

In [222]:
res.premise_area.describe()

count    231869.000000
mean         65.457164
std          88.241237
min           5.100000
25%          36.300000
50%          56.300000
75%          75.900000
max        9471.500000
Name: premise_area, dtype: float64

In [173]:
build[build['premise_floor_count']=='1,2,9']

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
45458,"1945-1959,Pre 1919",NaN,"Office,Retail",None,"1,2,9",35.2,2521.1,2521.1,986.5,22689.9,...,33739665,1.009119e+10,23,0.0,osgb1000006532203,Complex age,Complex floors,Complex use,Non Residential,"MULTIPOLYGON (((537818.980 189720.470, 537819...."


In [170]:
df = pd.read_csv('/Users/gracecolverd/New_dataset/data/mappings/avg_heights_all_buildings_uk.csv')
df= df[df['map_simple_use']=='Residential'] 
df['premise_floor_count'] = df['premise_floor_count'].astype(int)

In [171]:
df

,Unnamed: 0,map_simple_use,premise_age,premise_floor_count,weighted_average_height
5206,5206,Residential,1837-1869,1,7.402269
5207,5207,Residential,1837-1869,10,14.100000
5208,5208,Residential,1837-1869,11,22.480000
5209,5209,Residential,1837-1869,12,6.400000
5210,5210,Residential,1837-1869,16,0.000000
...,...,...,...,...,...
5536,5536,Residential,Unknown date,5,11.306400
5537,5537,Residential,Unknown date,6,12.734641
5538,5538,Residential,Unknown date,7,17.894915
5539,5539,Residential,Unknown date,8,22.325000


In [168]:
df.columns

Index(['Unnamed: 0', 'map_simple_use', 'premise_age', 'premise_floor_count',
       'weighted_average_height'],
      dtype='object')

In [156]:
df = fill_premise_floor_types(build)

starting flood replace


ValueError: could not convert string to float: 'Undetermined'

In [137]:
df.premise_floor_count.value_counts()

2     245099
1      82878
3      21839
0      14988
4       6812
5        194
6         48
7         22
10        11
12         7
8          6
11         6
9          5
13         3
15         2
16         1
18         1
22         1
14         1
26         1
19         1
53         1
Name: premise_floor_count, dtype: Int64

In [119]:
len(build[build['premise_use']=='Residential']  ), len(merg)

(291361, 277244)

In [109]:
build[build['premise_type']=='Residential'].toid.isna().sum()

0

In [100]:
build[build['verisk_building_id']==43004545].groupby('toid').count().sort_values('premise_age')

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,distance_water,verisk_building_id,uprn,uprn_count,uprn_distance,map_age,map_floors,map_use,map_simple_use,geometry
toid,,,,,,,,,,,,,,,,,,,,,
osgb1000011183015,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
osgb1000011183248,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
osgb1000011183251,1,0,1,0,0,1,1,1,0,0,...,1,1,1,1,1,1,0,1,1,1
osgb1000011183253,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
osgb1000011183254,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
osgb1000011183213,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
osgb1000011183207,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
osgb1000011183206,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [83]:
# Merge the average heights back into the original dataframe
df = pd.merge(df, avg_heights, on=['premise_age', 'premise_floor_count', 'map_simple_use'], how='left')

# Update 'height' where it is 0 with the 'avg_height' from the same 'premise_age' and 'premise_floor_count' combination
df['height'] = np.where(df['height'] == 0, df['avg_height'], df['height'])

# Drop the 'avg_height' column as it's no longer needed
# df.drop('avg_height', axis=1, inplace=True)

In [84]:
df[['height', 'avg_height']]

,height,avg_height
0,8.5,8.302126
1,8.4,8.302126
2,8.3,7.953602
3,9.2,8.698319
4,8.6,8.370757
...,...,...
291356,9.2,8.261101
291357,9.3,8.383414
291358,8.9,8.383414
291359,8.8,8.383414


In [12]:
b = build[build['height']==0][build['premise_use']=='Residential'].iloc[0]

/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/geopandas/geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [15]:
build['']

/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/pandas/core/dtypes/inference.py:387: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/pandas/core/dtypes/inference.py:388: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  len(obj)  # Has a length associated with it.
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/pandas/io/formats/printing.py:117: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi

premise_age                                                       Post 1999
premise_year                                                         2021.0
premise_use                                                     Residential
premise_type                                         Standard size detached
premise_floor_count                                                       2
height                                                                  0.0
premise_area                                                          105.0
building_area                                                         105.0
address_area                                                          210.0
gross_area                                                            210.0
basement                                                  Basement unlikely
listed_grade                                                           None
element_count                                                             1
bathroom_cou

In [14]:
b['premise_area']

105.0

In [13]:
b.geometry.area

105.03080000006493

In [7]:
# where basemenet =='confirmed' add one extra flooor 
build['base_floor'] = build['basement'].apply(lambda x: 1 if ((x == 'Basement confirmed') or (x=='Basement likely')) else 0)
# data.columns

In [10]:
build[build['base_floor']!=0]['basement']

97        Basement confirmed
3822      Basement confirmed
4404      Basement confirmed
4654      Basement confirmed
5711      Basement confirmed
                 ...        
373724    Basement confirmed
373737       Basement likely
373742    Basement confirmed
373937    Basement confirmed
373994       Basement likely
Name: basement, Length: 839, dtype: object

In [3]:
data =build[~build['uprn'].isna()]
res = data[data['map_simple_use']=='Residential']

In [14]:
build['glazing_type'].value_counts()

double glazing installed before 2002             48263
double glazing installed during or after 2002    42511
double glazing, unknown install date             39989
NO DATA!                                         12067
not defined                                       5195
secondary glazing                                 1401
single glazing                                     650
INVALID!                                           481
triple glazing                                     230
double, known data                                  27
triple, known data                                   9
Name: glazing_type, dtype: int64

In [7]:
build.columns

Index(['premise_age', 'premise_year', 'premise_use', 'premise_type',
       'premise_floor_count', 'height', 'premise_area', 'building_area',
       'address_area', 'gross_area', 'basement', 'listed_grade',
       'element_count', 'bathroom_count', 'bedroom_count',
       'reception_room_count', 'roof_type', 'wall_type', 'substructure_type',
       'glazing_type', 'wall_construction_type', 'extension_count',
       'habitable_rooms', 'open_fireplaces', 'floor_type', 'distance_building',
       'site_id', 'site_area', 'site_non_built_area', 'site_building_count',
       'distance_water', 'verisk_building_id', 'uprn', 'uprn_count',
       'uprn_distance', 'toid', 'map_age', 'map_floors', 'map_use',
       'map_simple_use', 'geometry'],
      dtype='object')

In [13]:
build.head()

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
0,Unknown date,NaN,Industry,None,None,8.3,155.0,155.0,NaN,NaN,...,38261175,NaN,0,NaN,None,Unknown date,None,Industry,Non Residential,"MULTIPOLYGON (((509255.552 319987.041, 509246...."
1,1870-1918,NaN,Industry,None,None,15.8,1091.0,1091.0,NaN,NaN,...,38261431,NaN,0,NaN,None,1870-1918,None,Industry,Non Residential,"MULTIPOLYGON (((509207.959 320049.532, 509220...."
2,1919-1944,NaN,Residential,Standard size detached,2,8.5,82.8,82.8,165.6,165.6,...,38261435,1.000309e+11,1,0.0,osgb1000010970795,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509310.990 320035.572, 509303...."
3,1870-1918,NaN,Industry,None,None,15.1,1197.3,4048.9,NaN,NaN,...,38261436,1.000322e+11,1,0.0,osgb1000010970781,1870-1918,None,Industry,Non Residential,"MULTIPOLYGON (((509257.259 320053.539, 509255...."
4,1919-1944,NaN,Residential,Standard size detached,2,8.4,88.5,88.5,177.0,177.0,...,38261443,1.000309e+11,1,0.0,osgb1000010970798,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509316.695 320054.606, 509308...."


In [16]:
data

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
2,1919-1944,NaN,Residential,Standard size detached,2,8.5,82.8,82.8,165.6,165.6,...,38261435,1.000309e+11,1,0.0,osgb1000010970795,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509310.990 320035.572, 509303...."
3,1870-1918,NaN,Industry,None,None,15.1,1197.3,4048.9,NaN,NaN,...,38261436,1.000322e+11,1,0.0,osgb1000010970781,1870-1918,None,Industry,Non Residential,"MULTIPOLYGON (((509257.259 320053.539, 509255...."
4,1919-1944,NaN,Residential,Standard size detached,2,8.4,88.5,88.5,177.0,177.0,...,38261443,1.000309e+11,1,0.0,osgb1000010970798,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509316.695 320054.606, 509308...."
6,1870-1918,NaN,Residential,Small low terraces,2,9.2,62.9,652.9,125.8,125.8,...,38261448,1.000309e+11,1,0.0,osgb1000010970810,1870-1918,2,Residential,Residential,"MULTIPOLYGON (((509347.023 320108.769, 509349...."
7,Pre 1919,NaN,Residential,Small low terraces,2,8.6,168.2,438.2,336.4,336.4,...,38261447,1.000309e+11,1,0.0,osgb1000010970818,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509316.153 320117.787, 509317...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374527,Post 1999,2023.0,Unknown,None,None,NaN,74.9,74.9,NaN,NaN,...,44898744,1.009413e+10,1,0.0,osgb5000005305950191,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568022.280 341549.640, 568020...."
374528,Post 1999,2023.0,Unknown,None,None,NaN,79.0,79.0,NaN,NaN,...,44901587,1.009413e+10,1,0.0,osgb5000005305950221,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568046.700 341618.500, 568056...."
374529,Post 1999,2023.0,Unknown,None,None,NaN,78.7,78.7,NaN,NaN,...,44906389,1.009413e+10,2,0.0,osgb5000005305950183,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568048.680 341574.110, 568046...."
374531,Post 1999,2023.0,Unknown,None,None,NaN,78.5,78.5,NaN,NaN,...,44916858,1.009413e+10,1,0.0,osgb5000005305950192,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568028.660 341600.130, 568030...."


In [17]:
data.map_use.value_counts() 

Residential                           220570
Commercial - mixed use                  9432
Unknown                                 5554
Recreation and leisure                  4215
Agricultural                            2886
Retail                                  1589
Industry                                1055
Retail below office or residential      1044
Religious                                982
Educational                              605
Office                                   243
Health                                   139
Institutional accommodation              126
Transport                                 92
Emergency services                        81
Governmental                              61
Utilities                                 55
Petrol station                            32
Commercial - derelict                     31
Residential with retail below             26
Retail under office                       18
Warehousing                               12
Retail wit

In [21]:

data[data['map_use']=='Unknown'] 
print('all Unknown in use as under non residential simple use ) ')

all Unknown in use as under non residential) 


In [18]:
data.map_simple_use.value_counts() 

Residential        220570
Non Residential     26472
Mixed Use            1813
Name: map_simple_use, dtype: int64

In [27]:
# basemenet attr seems flly populated for residentail builds 
res['basement'].isna().sum()

0

In [28]:
res[['premise_age', 'premise_year', 'map_age']]

,premise_age,premise_year,map_age
2,1919-1944,NaN,1919-1944
4,1919-1944,NaN,1919-1944
6,1870-1918,NaN,1870-1918
7,Pre 1919,NaN,Pre 1919
8,Pre 1919,NaN,Pre 1919
...,...,...,...
374486,1945-1959,NaN,1945-1959
374487,Post 1999,2011.0,Post 1999
374488,Post 1999,2011.0,Post 1999
374489,Post 1999,2011.0,Post 1999


In [62]:
data = build[( build['premise_use']=='Residential') &(  build['premise_age']!='Unknown date')].copy() 
data  = data.groupby(['premise_type', 'premise_age'])['verisk_building_id'].count().reset_index() 

In [63]:
data

,premise_type,premise_age,verisk_building_id
0,2 storeys terraces with t rear extension,1837-1869,919
1,2 storeys terraces with t rear extension,1870-1918,54179
2,2 storeys terraces with t rear extension,1919-1944,114
3,2 storeys terraces with t rear extension,1945-1959,11
4,2 storeys terraces with t rear extension,1960-1979,14
...,...,...,...
110,Very tall point block flats,1945-1959,5
111,Very tall point block flats,1960-1979,78
112,Very tall point block flats,1980-1989,3
113,Very tall point block flats,1990-1999,1


In [61]:
res['premise_age'].value_counts()

1960-1979       48577
1945-1959       34668
Post 1999       32936
1919-1944       29932
1870-1918       24850
1990-1999       21159
1980-1989       16328
Pre 1919         8207
Pre 1837         2024
Unknown date     1298
1837-1869         591
Name: premise_age, dtype: int64

In [67]:
# if building missing premise age, take mode of rest of postcode 

if len(data[data['premise_age'] =='Unknown date'] ) / len(data) * 100  < 10:
    print('less than 10% of the data is missing premise age')
    print('filling with mode of the rest of the data')
    data['premise_age2'] = data['premise_age'].replace('Unknown date', data['premise_age'].mode()[0])    


2.399389202547668

In [72]:
data['premise_age'].mode()[0]


'1960-1979'

In [69]:
data['premise_age'].replace('Unknown date', data['premise_age'].mode()[0])    

2         1919-1944
3         1870-1918
4         1919-1944
6         1870-1918
7          Pre 1919
            ...    
374527    Post 1999
374528    Post 1999
374529    Post 1999
374531    Post 1999
374533    Post 1999
Name: premise_age, Length: 248855, dtype: object

In [83]:
res['premise_age'].unique()

array(['1919-1944', '1870-1918', 'Pre 1919', '1960-1979', '1980-1989',
       '1945-1959', '1990-1999', 'Pre 1837', 'Post 1999', 'Unknown date',
       '1837-1869'], dtype=object)

In [ ]:
[
'Pre 1919',
'1919-1944',
'1945-1959', 
'1960-1979',
'1980-1989',
'1990-1999',
'Post 1999' 
]
       
       

In [87]:
# Map the categories to their ordinal number based on category_order


{'Pre 1919': 0,
 '1919-1944': 1,
 '1945-1959': 2,
 '1960-1979': 3,
 '1980-1989': 4,
 '1990-1999': 5,
 'Post 1999': 6}

In [88]:
data

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
2,1919-1944,NaN,Residential,Standard size detached,2,8.5,82.8,82.8,165.6,165.6,...,38261435,1.000309e+11,1,0.0,osgb1000010970795,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509310.990 320035.572, 509303...."
3,1870-1918,NaN,Industry,None,None,15.1,1197.3,4048.9,NaN,NaN,...,38261436,1.000322e+11,1,0.0,osgb1000010970781,1870-1918,None,Industry,Non Residential,"MULTIPOLYGON (((509257.259 320053.539, 509255...."
4,1919-1944,NaN,Residential,Standard size detached,2,8.4,88.5,88.5,177.0,177.0,...,38261443,1.000309e+11,1,0.0,osgb1000010970798,1919-1944,2,Residential,Residential,"MULTIPOLYGON (((509316.695 320054.606, 509308...."
6,1870-1918,NaN,Residential,Small low terraces,2,9.2,62.9,652.9,125.8,125.8,...,38261448,1.000309e+11,1,0.0,osgb1000010970810,1870-1918,2,Residential,Residential,"MULTIPOLYGON (((509347.023 320108.769, 509349...."
7,Pre 1919,NaN,Residential,Small low terraces,2,8.6,168.2,438.2,336.4,336.4,...,38261447,1.000309e+11,1,0.0,osgb1000010970818,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509316.153 320117.787, 509317...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374527,Post 1999,2023.0,Unknown,None,None,NaN,74.9,74.9,NaN,NaN,...,44898744,1.009413e+10,1,0.0,osgb5000005305950191,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568022.280 341549.640, 568020...."
374528,Post 1999,2023.0,Unknown,None,None,NaN,79.0,79.0,NaN,NaN,...,44901587,1.009413e+10,1,0.0,osgb5000005305950221,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568046.700 341618.500, 568056...."
374529,Post 1999,2023.0,Unknown,None,None,NaN,78.7,78.7,NaN,NaN,...,44906389,1.009413e+10,2,0.0,osgb5000005305950183,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568048.680 341574.110, 568046...."
374531,Post 1999,2023.0,Unknown,None,None,NaN,78.5,78.5,NaN,NaN,...,44916858,1.009413e+10,1,0.0,osgb5000005305950192,Post 1999,None,Unknown,Non Residential,"MULTIPOLYGON (((568028.660 341600.130, 568030...."


In [84]:
res[res['premise_age']=='Pre 1919']

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
7,Pre 1919,NaN,Residential,Small low terraces,2,8.6,168.2,438.2,336.4,336.4,...,38261447,1.000309e+11,1,0.0,osgb1000010970818,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509316.153 320117.787, 509317...."
8,Pre 1919,NaN,Residential,Small low terraces,2,9.2,49.9,652.9,99.8,99.8,...,38261448,1.000309e+11,1,0.0,osgb1000010970811,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509354.075 320123.800, 509355...."
9,Pre 1919,NaN,Residential,Small low terraces,2,8.6,110.4,438.2,220.8,220.8,...,38261447,1.000309e+11,1,0.0,osgb1000010970819,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509316.153 320117.787, 509311...."
13,Pre 1919,NaN,Residential,Standard size semi detached,2,6.8,42.3,84.6,84.6,84.6,...,38261454,1.000309e+11,1,0.0,osgb1000010970685,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((509260.096 320156.463, 509265...."
16,Pre 1919,NaN,Residential,3-4 storey and smaller flats,4,10.8,225.1,896.3,300.1,900.4,...,38261467,2.000028e+11,3,0.0,osgb1000010970838,Pre 1919,4,Residential,Residential,"MULTIPOLYGON (((509385.000 320128.595, 509374...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374064,Pre 1919,NaN,Residential,Standard size detached,2,7.7,142.1,142.1,284.2,284.2,...,42893463,1.000910e+11,1,0.0,osgb1000011424596,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((567424.930 341374.670, 567427...."
374111,Pre 1919,NaN,Residential,Standard size semi detached,2,9.0,179.9,179.9,179.9,359.8,...,42893444,1.000910e+11,2,0.0,osgb1000011424452,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((567671.340 341336.800, 567668...."
374137,Pre 1919,NaN,Residential,Standard size semi detached,2,9.0,194.8,194.8,129.9,389.6,...,42893484,1.000910e+11,3,0.0,osgb1000011424147,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((567509.540 341505.950, 567508...."
374162,Pre 1919,NaN,Residential,Standard size semi detached,2,8.9,131.5,131.5,131.5,263.0,...,42893497,1.000910e+11,2,0.0,osgb1000011424169,Pre 1919,2,Residential,Residential,"MULTIPOLYGON (((567594.370 341475.650, 567602...."


In [62]:
res[res['premise_age']=='Unknown date']

,premise_age,premise_year,premise_use,premise_type,premise_floor_count,height,premise_area,building_area,address_area,gross_area,...,verisk_building_id,uprn,uprn_count,uprn_distance,toid,map_age,map_floors,map_use,map_simple_use,geometry
862,Unknown date,NaN,Residential,None,2,8.6,132.7,132.7,265.4,265.4,...,38263640,1.000323e+11,1,0.00,osgb1000010977171,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((510094.615 319757.165, 510081...."
1988,Unknown date,NaN,Residential,None,2,9.2,325.8,325.8,651.6,651.6,...,38256952,1.000728e+10,1,2.02,osgb5000005288478281,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((509559.870 316759.550, 509548...."
2225,Unknown date,NaN,Residential,None,2,8.7,269.6,269.6,539.2,539.2,...,38257630,1.000728e+10,1,0.00,osgb1000010967620,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((509691.630 317592.510, 509702...."
2583,Unknown date,NaN,Residential,None,2,9.4,71.6,71.6,71.6,143.2,...,38255945,1.000728e+10,2,0.00,osgb1000010975899,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((510338.370 314422.030, 510340...."
2898,Unknown date,NaN,Residential,None,1,3.0,41.3,41.3,41.3,41.3,...,38257194,1.000728e+10,1,0.00,osgb1000010966950,Unknown date,1,Residential,Residential,"MULTIPOLYGON (((509456.640 316975.930, 509460...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368161,Unknown date,NaN,Residential,3-4 storey and smaller flats,2,9.1,96.0,96.0,192.0,192.0,...,42568396,1.000916e+11,1,0.00,osgb1000002673089035,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((581327.550 309200.810, 581328...."
368162,Unknown date,NaN,Residential,Standard size detached,2,7.8,76.9,76.9,153.8,153.8,...,42568398,1.001199e+10,1,0.00,osgb1000011459698,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((581343.120 309211.070, 581330...."
370014,Unknown date,NaN,Residential,Standard size detached,2,8.1,88.4,88.4,88.4,176.8,...,42897726,1.009413e+10,2,0.00,osgb5000005294677799,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((582931.220 342049.860, 582928...."
371392,Unknown date,NaN,Residential,None,2,7.5,408.4,408.4,816.8,816.8,...,42621520,1.000913e+11,1,0.00,osgb1000011478967,Unknown date,2,Residential,Residential,"MULTIPOLYGON (((588512.800 317593.420, 588504...."


In [94]:
import pandas as pd 
import geopandas as gpd 

import geopandas as gpd
import glob 
import pandas as pd
import os
import concurrent.futures
from osgeo import ogr
import pandas as pd
import os
import shutil 
import tempfile
import threading


import sys 
sys.path.append('/Users/gracecolverd/New_dataset')
from src.buildings import get_bounding_boxes , calculate_bounding_boxes, generate_new_filename



year = 2022
input_gpk = '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_22_updated_data/UKBuildings_Edition_15_new_format_upn.gpkg' 


ds = ogr.Open(input_gpk)
layer = ds.GetLayer()
extent = layer.GetExtent()
bounding_boxes = calculate_bounding_boxes(extent, chunk_height=100000, chunk_width=100000)
len(bounding_boxes)


91

In [49]:
import geopandas as gpd
import pandas as pd
from scipy.stats import mode



def test_medheights(bbox_list , input_gpk ):
    # Placeholder for aggregated statistics and weights

    max_file = []
    for bbox in bbox_list:
        print('Starting ', bbox )
        subset = gpd.read_file(input_gpk, bbox=bbox)
        if subset.empty:
            print('Empty subset')
            continue
        max_file.append(subset)
    max_ds = pd.concat(max_file)
    return max_ds




In [66]:
data.groupby(['premise_type', 'premise_age']).sum('verisk_building_id').reset_index()

,premise_type,premise_age,verisk_building_id
0,2 storeys terraces with t rear extension,1837-1869,919
1,2 storeys terraces with t rear extension,1870-1918,54179
2,2 storeys terraces with t rear extension,1919-1944,114
3,2 storeys terraces with t rear extension,1945-1959,11
4,2 storeys terraces with t rear extension,1960-1979,14
...,...,...,...
110,Very tall point block flats,1945-1959,5
111,Very tall point block flats,1960-1979,78
112,Very tall point block flats,1980-1989,3
113,Very tall point block flats,1990-1999,1


In [82]:
df= build.copy() 

data = df[( df['premise_use']=='Residential') &(  df['premise_age']!='Unknown date')].copy() 
data.groupby(['premise_use', 'premise_type', 'premise_age']).size().reset_index().rename(columns={0:'count'})

,premise_use,premise_type,premise_age,count
0,Residential,2 storeys terraces with t rear extension,1837-1869,919
1,Residential,2 storeys terraces with t rear extension,1870-1918,54179
2,Residential,2 storeys terraces with t rear extension,1919-1944,114
3,Residential,2 storeys terraces with t rear extension,1945-1959,11
4,Residential,2 storeys terraces with t rear extension,1960-1979,14
...,...,...,...,...
110,Residential,Very tall point block flats,1945-1959,5
111,Residential,Very tall point block flats,1960-1979,78
112,Residential,Very tall point block flats,1980-1989,3
113,Residential,Very tall point block flats,1990-1999,1


In [92]:
import geopandas as gpd
import pandas as pd
from scipy.stats import mode

def compute_global_modal_age(bbox_list, input_gpk):
    # Function to find the modal premise age for each premise_type

    def find_modal_ages(df):
        # Group by 'premise_use', 'premise_type' and find the index of the maximum 'verisk_building_id' for each group
        idx = df.groupby(['premise_use', 'premise_type'])['count'].idxmax()
        
        # Use the index to select the rows corresponding to the modal ages
        modes_df = df.loc[idx].reset_index(drop=True)
        
        # Select only the columns of interest
        modes_df = modes_df[['premise_use', 'premise_type', 'premise_age', 'count']]

        return modes_df

    def get_age_counts(df):
        data = df[( df['premise_use']=='Residential') &(  df['premise_age']!='Unknown date')].copy() 
        return data.groupby(['premise_use', 'premise_type', 'premise_age']).size().reset_index().rename(columns={0:'count'})
    # Dictionary to accumulate ages for each group
    age_data = {'premise_use':[], 'premise_type': [], 'premise_age': [], 	'count': [] } 

    for bbox in bbox_list:
        print('starting bbox')
        # Read subset within the current bbox
        subset = gpd.read_file(input_gpk, bbox=bbox)
        
        # Filter out 'Unknown date'
        subset = subset[(subset['premise_age'] != 'Unknown date') & (subset['premise_use']=='Residential')]

        # Skip empty subsets
        if subset.empty:
            continue
        print('calc subset')
        subset = get_age_counts(subset)
        for col in age_data.keys():
            age_data[col].extend(subset[col].tolist())   

    print('starting union')
    age_df = pd.DataFrame(age_data)
    age_df= age_df.groupby(['premise_use', 'premise_type', 'premise_age']).sum('count').reset_index()

    age_df = find_modal_ages(age_df)

    return age_df

# Note: Ensure bbox_list and input_gpk are correctly defined before calling this function.


In [96]:
ds = ogr.Open(input_gpk)
layer = ds.GetLayer()
extent = layer.GetExtent()
bounding_boxes = calculate_bounding_boxes(extent, chunk_height=10000, chunk_width=10000)
bbox_test = bounding_boxes[0:1000]

compute_global_modal_age(bbox_test, input_gpk)

starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starti

,premise_use,premise_type,premise_age,count
0,Residential,3-4 storey and smaller flats,Post 1999,10
1,Residential,Large detached,Post 1999,250
2,Residential,Large semi detached,Post 1999,3
3,Residential,Small low terraces,1960-1979,46
4,Residential,Standard size detached,Post 1999,756
5,Residential,Standard size semi detached,1960-1979,118
6,Residential,Tall terraces 3-4 storeys,Pre 1837,5


In [97]:
test = test_medheights(bbox_test, input_gpk)

Starting  (9252.36, 6435.25920425788, 19252.36, 16435.25920425788)
Empty subset
Starting  (9252.36, 16435.25920425788, 19252.36, 26435.25920425788)
Empty subset
Starting  (9252.36, 26435.25920425788, 19252.36, 36435.259204257876)
Empty subset
Starting  (9252.36, 36435.259204257876, 19252.36, 46435.259204257876)
Empty subset
Starting  (9252.36, 46435.259204257876, 19252.36, 56435.259204257876)
Empty subset
Starting  (9252.36, 56435.259204257876, 19252.36, 66435.25920425788)
Empty subset
Starting  (9252.36, 66435.25920425788, 19252.36, 76435.25920425788)
Empty subset
Starting  (9252.36, 76435.25920425788, 19252.36, 86435.25920425788)
Empty subset
Starting  (9252.36, 86435.25920425788, 19252.36, 96435.25920425788)
Empty subset
Starting  (9252.36, 96435.25920425788, 19252.36, 106435.25920425788)
Empty subset
Starting  (9252.36, 106435.25920425788, 19252.36, 116435.25920425788)
Empty subset
Starting  (9252.36, 116435.25920425788, 19252.36, 126435.25920425788)
Empty subset
Starting  (9252.36

In [102]:
d= test.groupby(['premise_use', 'premise_type', 'premise_age']).size().reset_index()

In [107]:
idx = d.groupby(['premise_use', 'premise_type'])[0].idxmax()

In [93]:
test = bounding_boxes[0:25]

compute_global_modal_age(test, input_gpk)

starting bbox
calc subset
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
starting bbox
starting union


,premise_use,premise_type,premise_age,count
0,Residential,2 storeys terraces with t rear extension,1870-1918,4988
1,Residential,3-4 storey and smaller flats,1980-1989,1811
2,Residential,Large detached,Post 1999,5188
3,Residential,Large semi detached,1870-1918,1072
4,Residential,Linked and step linked premises,1960-1979,941
5,Residential,Medium height flats 5-6 storeys,Post 1999,64
6,Residential,Semi type house in multiples,1945-1959,1851
7,Residential,Small low terraces,1870-1918,21414
8,Residential,Standard size detached,1960-1979,22867
9,Residential,Standard size semi detached,1945-1959,15871


In [91]:
test = bounding_boxes[0:25]

compute_global_modal_age(test, input_gpk)

starting bbox
calc subset
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox
calc subset
starting bbox
starting bbox
starting bbox
calc subset
starting bbox
calc subset
starting bbox


KeyboardInterrupt: 

In [40]:
test = test_medheights(test, input_gpk)

Starting  (9252.36, 6435.25920425788, 109252.36, 106435.25920425788)
Starting  (9252.36, 106435.25920425788, 109252.36, 206435.25920425786)
Empty subset
Starting  (9252.36, 206435.25920425786, 109252.36, 306435.25920425786)
Empty subset
Starting  (9252.36, 306435.25920425786, 109252.36, 406435.25920425786)
Empty subset
Starting  (9252.36, 406435.25920425786, 109252.36, 506435.25920425786)
Empty subset
Starting  (9252.36, 506435.25920425786, 109252.36, 606435.2592042579)
Empty subset
Starting  (9252.36, 606435.2592042579, 109252.36, 706435.2592042579)
Empty subset
Starting  (9252.36, 706435.2592042579, 109252.36, 806435.2592042579)
Starting  (9252.36, 806435.2592042579, 109252.36, 906435.2592042579)
Starting  (9252.36, 906435.2592042579, 109252.36, 1006435.2592042579)
Starting  (9252.36, 1006435.2592042579, 109252.36, 1106435.2592042577)
Empty subset
Starting  (9252.36, 1106435.2592042577, 109252.36, 1206435.2592042577)
Empty subset
Starting  (9252.36, 1206435.2592042577, 109252.36, 121

In [41]:
# aapply mode fn 
test=test[test['premise_age']!='Unknown date']  
test.groupby(['premise_use', 'premise_type'])['premise_age'].apply(lambda x: mode(x).mode[0]).reset_index(name='mode_age')

/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_25760/3819135760.py:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  test.groupby(['premise_use', 'premise_type'])['premise_age'].apply(lambda x: mode(x).mode[0]).reset_index(name='mode_age')
/Users/gracecolverd/miniconda3/envs/gdal_3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_25760/3819135760.py:3: DeprecationWarning: Support for no

,premise_use,premise_type,mode_age
0,Agricultural,Standard size detached,1870-1918
1,Commercial - derelict,Linked and step linked premises,1919-1944
2,Commercial - mixed use,Medium height flats 5-6 storeys,Pre 1919
3,Educational,2 storeys terraces with t rear extension,1870-1918
4,Educational,Small low terraces,1870-1918
5,Educational,Standard size detached,1870-1918
6,"Health,Institutional accommodation,Retail belo...",Small low terraces,Pre 1919
7,Office,3-4 storey and smaller flats,1919-1944
8,Office,Standard size detached,Pre 1919
9,Recreation and leisure,3-4 storey and smaller flats,1919-1944


In [45]:
test[test['premise_use']=='Agricultural']['premise_type'].unique()

array([None, 'Standard size detached'], dtype=object)

In [46]:
test.columns

Index(['premise_age', 'premise_year', 'premise_use', 'premise_type',
       'premise_floor_count', 'height', 'premise_area', 'building_area',
       'address_area', 'gross_area', 'basement', 'listed_grade',
       'element_count', 'bathroom_count', 'bedroom_count',
       'reception_room_count', 'roof_type', 'wall_type', 'substructure_type',
       'glazing_type', 'wall_construction_type', 'extension_count',
       'habitable_rooms', 'open_fireplaces', 'floor_type', 'distance_building',
       'site_id', 'site_area', 'site_non_built_area', 'site_building_count',
       'distance_water', 'verisk_building_id', 'uprn', 'uprn_count',
       'uprn_distance', 'toid', 'map_age', 'map_floors', 'map_use',
       'map_simple_use', 'upn', 'geometry'],
      dtype='object')